**Plot Azimuth, Elevation and Alt (height) in log file using plotly**

*Requirements:*
- pandas
- numpy
- plotly

In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [2]:
bdc_log_path = "/mnt/data/hungnv152/data/UAV_2X/bia_video/bia_den_tam_trang/simple/T12B/T12B/video_record/00000012.csv"
bdc_log = pd.read_csv(bdc_log_path)

frameId, imageId, alt, APTime, MCDur = [], [], [], [], []
roll, pitch, yaw, azi, ele = [], [], [], [], []
count = 1
lastAlt = -1
for id, row in bdc_log.iterrows():
    if row[' AbservationMode'] != ' Free':
        frameId.append(count)
        imageId.append(int(row['ImageId']))
        alt.append(float(row[' Alt']))
        APTime.append(int(row[' APTime']))
        MCDur.append(int(row[' MCDuration']))
        roll.append(float(row[' Roll']))
        pitch.append(float(row[' Pitch']))
        yaw.append(float(row[' Yaw']))
        ele.append(float(row[' Elevation']))
        azi.append(float(row[' Azimuth']))
        count += 1

meanAPTime = np.array(APTime).mean()
apPeaks = np.where(np.array(APTime > meanAPTime))[0]
APTimePeaks = np.array(APTime)[apPeaks]
meanMCDur = np.array(MCDur).mean()
mcPeaks = np.where(np.array(MCDur > meanMCDur))[0]
MCDurPeaks = np.array(MCDur)[mcPeaks]

dAlt = np.array(alt[1:]) - np.array(alt[:-1])
indices = np.where(dAlt > 100)[0].tolist()

localMins = []
localMaxs = []
for id in indices:
    localMinId = np.argmin(np.array(alt)[max(0, id - 100): min(len(alt), id + 100)]) + max(0, id - 100)
    localMins.append((frameId[localMinId], alt[localMinId], imageId[localMinId]))
    
    localMaxId = np.argmax(np.array(alt)[max(0, id - 100): min(len(alt), id + 100)]) + max(0, id - 100)
    localMaxs.append((frameId[localMaxId], alt[localMaxId], imageId[localMaxId]))
localMins.append((frameId[-1],alt[-1], imageId[-1]))
localMaxs.append((frameId[0],alt[0], imageId[0]))


In [3]:
pio.renderers.default = 'vscode' 

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=frameId, y=np.degrees(np.array(ele)),
    mode='lines',
    line=dict(color='blue', width=2),
    name='Elevation',
    customdata = imageId,
    hovertemplate='Frame: %{customdata}<br>Ele: %{y}°<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=frameId, y=np.degrees(np.array(azi)),
    mode='lines',
    line=dict(color='red', width=2),
    name='Azimuth',
    customdata = imageId,
    hovertemplate='Frame: %{customdata}<br>Azi: %{y}°<extra></extra>',
    yaxis = 'y2'
))

fig.add_trace(go.Scatter(
    x=frameId, y=alt,
    mode='lines',
    line=dict(color='yellow', width=2),
    name='Alt',
    yaxis='y3',
    customdata = imageId,
    hovertemplate='Frame: %{customdata}<br>Alt: %{y}°<extra></extra>'
))

fig.update_layout(
    yaxis=dict(
        title='Ele (deg)',
        position = 0.0
    ),
    yaxis2=dict(
        title='Azi (deg)',
        side='left',
        overlaying = 'y',
    ),
    yaxis3=dict(
        title='Meter',
        overlaying='y',
        side='right'
    ),
    xaxis=dict(title='', domain = [0.05, 1.0]),
    legend=dict(x=0.5, y=1.15, xanchor='center', orientation='h'),
    # margin = dict(l = 200)
)

for point, peakValue in zip(apPeaks, APTimePeaks):  
    fig.add_vline(x = point, line=dict(color='green', width=1, dash='dash'), name = 'AP Time peak')

# for point, peakValue in zip(mcPeaks, MCDurPeaks):  
#     fig.add_vline(x = point, line=dict(color='magenta', width=2, dash='dash'), name = 'MC Duration peak')

fig.show()